In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from sklearn import tree, neural_network, neighbors, discriminant_analysis

%matplotlib inline
plt.rcParams['figure.figsize'] = (4, 4) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# for auto-reloading extenrnal modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

## Discrimination in a Single Dimension

### 1D Gaussian Discriminant 

In [ ]:
figure = plt.figure()
ax = plt.gca()

a = np.random.normal(loc = -1, scale = 2, size = 10000)
b = np.random.normal(loc = 3, scale = 1, size = 10000)

r = np.min([a.min(), b.min()]), np.max([a.max(), b.max()])

plt.hist([a, b], bins=30, range=r, histtype = 'step', color = ["b", "r"], label = ["Category 1", "Category 2"], alpha = 1.0)
plt.legend(loc = "upper left", frameon=False)

ax.set_xlabel("Gaussian Discriminants")
figure.savefig('gaus_discr.pdf', bbox_inches='tight', pad_inches=0.1)
plt.show()

### ROC Curve

In [ ]:
cut = np.linspace(-6, 6, 500)
eff = np.array([np.mean(a < c) for c in cut])
rej = np.array([np.mean(b > c) for c in cut])

figure = plt.figure()
ax = plt.gca()

plt.plot(eff, rej, c = "k")
ax.set_xlim(0, 1.07)
ax.set_ylim(0, 1.07)

ax.set_xlabel('Blue Efficiency', color = "b")
ax.set_ylabel('Red Rejection', color = "r")

plt.scatter([1], [1], c = "b", s = 200, marker = "*", linewidth = 0)

figure.savefig('gaus_roc.pdf', bbox_inches='tight', pad_inches=0.1)

plt.show()

## Plot Training Sample and Mesh

In [ ]:
def plot_classifier(Xtr, ytr, alpha = 0.2, alpha_pt = None, fn = None, h = 0.01, file = None):

    fig = plt.figure()
    
    if fn:
        
        xx, yy = np.meshgrid(np.arange(-1.3, 1.3, h),np.arange(-1.3, 1.3, h))
        Z = fn(np.c_[xx.ravel(), yy.ravel()])
        Z = Z.reshape(xx.shape)

        plt.contourf(xx, yy, Z, cmap=plt.cm.viridis, alpha = alpha)


    if alpha_pt: alpha = alpha_pt
    plt.scatter(Xtr[:, 0], Xtr[:, 1], c=ytr, s=40, cmap=plt.cm.viridis, alpha = alpha, linewidth = 0.1)
    
    plt.xlim([-1.3, 1.3])
    plt.ylim([-1.3, 1.3])
    
    if file: fig.savefig(file, bbox_inches='tight', pad_inches=0.1)


## Define a Spiral Dataset

In [ ]:
def spiral_data(K, N = 1000, D = 2, noise_level = 0.4):

    X = np.zeros((N*K,D))
    y = np.zeros(N*K, dtype='uint8')
    for j in range(K):

        ix = range(N*j,N*(j+1))

        spiral = np.linspace(0, 1.5 * np.pi,N)
        offset = 2 * np.pi * j / K
        noise  = np.random.randn(N)*noise_level

        theta  = spiral + noise + offset

        r = np.linspace(0.0,1,N) # radius    

        X[ix] = np.c_[r*np.sin(theta), r*np.cos(theta)]
        y[ix] = j # class
        
    return X, y

In [ ]:
X, y = spiral_data(K = 6, noise_level = 0.2)
plot_classifier(X, y)

## Define Blobs

In [ ]:
def blob_data(X = 2, Y = 2, N = 1000, D = 2, correlated = 0.5, noise_level = 1.0):

    K = X * Y
    
    par = np.zeros((N*K,D))
    cat = np.zeros(N*K, dtype='uint8')
    
    xstart = -0.5 if X > 1 else 0
    ystart = -0.5 if Y > 1 else 0
    
    xx, yy = np.meshgrid(np.linspace(xstart, 0.5, X), 
                         np.linspace(ystart, 0.5, Y))
    
    for j, (xctr,yctr) in enumerate(zip(xx.ravel(), yy.ravel())):

        ix = range(N*j,N*(j+1))

        corr = np.random.randn(N) * correlated 
        uncorr_x = np.random.randn(N) * (1 - correlated)
        uncorr_y = np.random.randn(N) * (1 - correlated)

        x = (corr + uncorr_x) * noise_level + xctr
        y = (corr + uncorr_y) * noise_level + yctr

        r = np.linspace(0.0,1,N) # radius    

        par[ix] = np.c_[x, y]
        cat[ix] = j 
        
    return par, cat

In [ ]:
X, y = blob_data(X = 5, Y = 1, noise_level = 0.15, correlated = 0.85)
plot_classifier(X, y)

## Principal Component Analysis and Linear Discriminant Analysis

In [ ]:
from sklearn import decomposition, discriminant_analysis

In [ ]:
X, y = blob_data(X = 1, Y = 1, noise_level = 0.35, correlated = 0.75)
y[X[:,0] > 0] = 1

plot_classifier(X, y)

In [ ]:
pca = decomposition.PCA(n_components = 2)
X_r = pca.fit(X).transform(X)
print('explained variance ratio (first two components): ' + str(pca.explained_variance_ratio_))

plt.figure()
plt.scatter(X_r[:, 0], X_r[:, 1], c = y, alpha=.8, cmap=plt.cm.viridis)
plt.show()

### Create a Series of Blobs, Overlapping in x and y.

In [ ]:
X, y = blob_data(X = 1, Y = 4, noise_level = 0.25, correlated = 0.8)
K = len(set(y))

cm = [plt.cm.viridis(v) for v in np.linspace(0, 1, K)]

In [ ]:
lda = discriminant_analysis.LinearDiscriminantAnalysis(n_components = 2)
lda.fit(X, y)

plot_classifier(X, y, fn = lda.predict, file = "lda_classifier.pdf")

In [ ]:
X_r2 = lda.fit(X, y)
X_r2 = lda.transform(X)

fig = plt.figure()
plt.scatter(X_r2[:, 0], X_r2[:, 1], c=y, s=40, cmap=plt.cm.viridis, alpha = 0.2, linewidth = 0.1)
fig.savefig('lda_transf.pdf', bbox_inches='tight', pad_inches=0.1)
plt.show()

fig = plt.figure()
ax = plt.gca()

ax.hist([X_r2[y == c, 0] for c in range(K)], 
        bins=30, histtype = 'step', color = cm, alpha = 1.0)
fig.savefig('lda_project.pdf', bbox_inches='tight', pad_inches=0.1)

plt.show()

In [ ]:
# Define blob data with 2×2
X, y = blob_data(X = 2, Y = 2, noise_level = 0.25, correlated = 0.8)
K = len(set(y))

cm = [plt.cm.viridis(v) for v in np.linspace(0, 1, K)]

# create the LDA and fit it to data
lda = discriminant_analysis.LinearDiscriminantAnalysis()
lda.fit(X, y)

print(lda.predict([[1, 1]]))

# plot it, using my function
plot_classifier(X, y, fn = lda.predict, file = "lda_classifier2.pdf")
X_r2 = lda.fit(X, y)
X_r2 = lda.transform(X)

fig = plt.figure()
plt.scatter(X_r2[:, 0], X_r2[:, 1], c=y, s=40, cmap=plt.cm.viridis, alpha = 0.2, linewidth = 0.1)
fig.savefig('lda_transf2.pdf', bbox_inches='tight', pad_inches=0.1)
plt.show()

fig = plt.figure()
ax = plt.gca()

ax.hist([X_r2[y == c, 0] for c in range(K)], 
        bins=30, histtype = 'step', color = cm, alpha = 1.0)
fig.savefig('lda_project2.pdf', bbox_inches='tight', pad_inches=0.1)

plt.show()

In [ ]:
X, y = spiral_data(K = 6, noise_level = 0.2)

lda = discriminant_analysis.LinearDiscriminantAnalysis(n_components = 2)
lda.fit(X, y)

plot_classifier(X[::5,], y[::5], fn = lda.predict, file = "lda_lollypop.pdf", alpha = 0.3)

## Nearest Neighbors and Overtraining

In [ ]:
X, y = blob_data(noise_level = 0.5, correlated = 0.5, N = 200)
# X, y = spiral_data(K = 7, N = 30, noise_level = 0.3)

In [ ]:
knn = neighbors.KNeighborsClassifier(1, weights="uniform")

knn.fit(X, y)
plot_classifier(X[:,], y[:], fn = knn.predict, alpha = 0.4, h = 0.01, file = "knn_overtrained.pdf")

knn = neighbors.KNeighborsClassifier(30, weights="uniform")

knn.fit(X, y)
plot_classifier(X[:,], y[:], fn = knn.predict, alpha = 0.4, h = 0.01, file = "knn_k30.pdf")

## In the real world...

You wouldn't write your own classifier (without a _lot_ more practice).

[scikit learn](http://scikit-learn.org/) implements many of the "standard" classification algorithms.  Let's see what these look like.  

BUT: _be careful!_  This is an incredibly powerful tool, and one that is very easy to misuse.  Check (but do not tune) to your test sample, use well-understood input variables.  And understand who/what you're selecting.

In [ ]:
X, y = blob_data(X = 2, Y = 2, N = 1000, noise_level = 0.5, correlated = 0.6)
# X, y = spiral_data(K = 5, noise_level = 0.4, N = 1000)

bdt = tree.DecisionTreeClassifier(min_samples_leaf = 1)
bdt.fit(X, y)

plot_classifier(X[::,], y[::], fn = bdt.predict, alpha = 0.4, alpha_pt = 0.06, file = "bdt_overtrained.pdf")

bdt = tree.DecisionTreeClassifier(min_samples_leaf = 15)
bdt.fit(X, y)

plot_classifier(X[::,], y[::], fn = bdt.predict, alpha = 0.4, alpha_pt = 0.06, file = "bdt_leaf15.pdf")


import pydotplus
from IPython.display import Image  
dot_data = tree.export_graphviz(bdt, out_file=None, filled=True, rounded=True, special_characters=True) 
graph = pydotplus.graph_from_dot_data(dot_data)  
graph.write_pdf("bdt_graph.pdf") 
Image(graph.create_png())  

In [ ]:
X, y = spiral_data(K = 5, noise_level = 0.4, N = 1000)

mlp = neural_network.MLPClassifier(solver='lbfgs', alpha = 1e0, random_state=1, hidden_layer_sizes=(100))
mlp.fit(X, y)

plot_classifier(X[:,], y[:], fn = mlp.predict, alpha = 0.4, alpha_pt = 0.05, file = "mlp_classifier.pdf")

In [ ]:
knn = neighbors.KNeighborsClassifier(10, weights="distance")
knn.fit(X, y)

plot_classifier(X, y, fn = knn.predict)

## Train a Linear Classifier

In [ ]:
X, y = blob_data(noise_level = 0.5, correlated = 0.5, N = 200)
# X, y = spiral_data(K = 5, noise_level = 0.3)

In [ ]:
# initialize parameters randomly
K = len(set(y))
D = X.shape[1]

W = 0.01 * np.random.randn(D,K)
b = np.zeros((1,K))

# some hyperparameters
step_size = 1e-1
reg = 1e-3 # regularization strength
epsilon = 1e-5

# gradient descent loop
num_examples = X.shape[0]
for i in range(5000): 
    
    # evaluate class scores, [N x K]
    scores = np.dot(X, W) + b 

    # compute the class probabilities
    exp_scores = np.exp(scores)
    probs = exp_scores / np.sum(exp_scores, axis=1, keepdims=True) # [N x K]

    # compute the loss: average cross-entropy loss and regularization
    correct_logprobs = -np.log(probs[range(num_examples),y])
    data_loss = np.sum(correct_logprobs)/num_examples
    reg_loss = 0.5*reg*np.sum(W*W)
    loss = data_loss + reg_loss
    
    if i % 1000 == 0: print("iteration {:d}: loss {:f}".format(i, loss))

    # compute the gradient on scores
    dscores = probs
    dscores[range(num_examples),y] -= 1
    dscores /= num_examples

    # backpropate the gradient to the parameters (W,b)
    dW = np.dot(X.T, dscores) + reg * W
    db = np.sum(dscores, axis=0, keepdims=True)

    # perform a parameter update
    W += -step_size * dW
    b += -step_size * db

    
def linear_classifier(X):
    
    # Calculate the max classifier value.
    return np.argmax(np.dot(X, W) + b, axis = 1)   
    

In [ ]:
print('Accuracy: {:.1f}%'.format(100. *np.mean(linear_classifier(X) == y)))
plot_classifier(X, y, fn = linear_classifier, alpha = 0.3)

## Build a Neural Network "By Hand"
Example adapted from:
* http://cs231n.github.io/neural-networks-case-study/
* http://www.wildml.com/2015/09/implementing-a-neural-network-from-scratch/

(Which have the same form...)

In [ ]:
import numpy as np

# Single hidden layer
class my_nn():
    
    def __init__(self, classes, dim, hidden = 100, step_size = 1, regularization = 1e-3):
        
        # dimensions
        self.h = hidden
        self.K = classes
        self.D = dim

        # randomly initialization
        self.W1 = 0.01 * np.random.randn(self.D, self.h)
        self.b1 = np.zeros((1, self.h))
        self.W2 = 0.01 * np.random.randn(self.h, self.K)
        self.b2 = np.zeros((1, self.K))

        # some hyperparameters
        self.step_size = step_size 
        self.regularization = regularization
        
    
    def evaluate(self, X):
        
        # Useful to save the hidden layer, for back propagation.
        self.hidden_layer = np.maximum(0, np.dot(X, self.W1) + self.b1) # ReLU activation
        scores = np.dot(self.hidden_layer, self.W2) + self.b2
        
        return scores
 
    def classify(self, X):
        
        return np.argmax(self.evaluate(X), axis=1)

    
    def train(self, X, iterations = 500):
        
        N = X.shape[0]

        # gradient descent loop
        for it in range(iterations):

            # evaluate 
            scores = self.evaluate(X)
            
            # compute the class probabilities
            exp_scores = np.exp(scores)
            probs = exp_scores / np.sum(exp_scores, axis=1, keepdims=True) # [N x K]

            # compute the loss: average cross-entropy loss and regularization
            corect_logprobs = -np.log(probs[range(N),y])
            data_loss = np.sum(corect_logprobs)/N
            reg_loss = 0.5 * self.regularization * (np.sum(self.W1*self.W1) + np.sum(self.W2*self.W2))
            loss = data_loss + reg_loss

            if it % 1000 == 0: print("iteration {:d}: loss {:.4f}".format(it, loss))

            # compute the gradient on scores
            dscores = probs
            dscores[range(N),y] -= 1
            dscores /= N

            # backpropate the gradient to the parameters
            # first backprop into parameters W2 and b2
            dW2 = np.dot(self.hidden_layer.T, dscores)
            db2 = np.sum(dscores, axis=0, keepdims=True)
            
            # backprop into hidden layer
            dhidden = np.dot(dscores, self.W2.T)
            # backprop the ReLU non-linearity
            dhidden[self.hidden_layer <= 0] = 0
            
            # finally into W1, b1
            dW1 = np.dot(X.T, dhidden)
            db1 = np.sum(dhidden, axis=0, keepdims=True)

            # add regularization gradient contribution
            # since we used λ W² / 2, it's just W.
            dW2 += self.regularization * self.W2
            dW1 += self.regularization * self.W1

            # perform a parameter update
            # step_size = 10 ** (-np.log(it+1)/np.log(5000))
            self.W1 += - self.step_size * dW1
            self.b1 += - self.step_size * db1
            self.W2 += - self.step_size * dW2
            self.b2 += - self.step_size * db2


In [ ]:
X, y = spiral_data(K = 5, noise_level = 0.3)

In [ ]:
nn = my_nn(classes = len(set(y)), dim = X.shape[1], hidden = 100)
nn.train(X, 5000)

In [ ]:
# nn.step_size = 0.1
# nn.train(X, 500)
print('Accuracy: {:.1f}%'.format(100. * np.mean(nn.classify(X) == y)))
plot_classifier(X[::5,], y[::5], fn = nn.classify, file = "my_nn_classifier.pdf", alpha = 0.3)